# Project Name

## Goal:
- Objective 1
- Objective 2

# Imports

In [29]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import wrangle as w
# import explore as e
# import model as m
# import evaluate as ev

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import unicodedata
import nltk
from wordcloud import WordCloud
import os
import json

from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr, f_oneway, ttest_ind

np.random.seed(123)

# Acquire
- Data acquired from _____ (e.g. [data.world](https://data.world/food/wine_quality) )
- It contained xx rows and xx columns before cleaning
- Each row represents ____
- Each column represents ____

# Prepare
- Removed ...
- Renamed ...
- Checked for nulls ...
- Checked that column data types were appropriate ...
- Outliers ?
- Added target column?
- Added additional features?
- Encoded categorical variables
- Split data into train, validate, and test (60/20/20), stratifying ...
- Scaled continuous variables


# Data Dictionary

| Feature | Type | Definition |
|:--------|:-----|:-----------|
|feature_1 (target)|integer|definition 1|

In [63]:
def split_data(df, target_variable):
    '''
    Takes in two arguments the dataframe name and the ("target_variable" - must be in string format) to stratify  and 
    return train, validate, test subset dataframes will output train, validate, and test in that order.
    '''
    train, test = train_test_split(df, #first split
                                   test_size=.2, 
                                   random_state=123, 
                                   stratify= df[target_variable])
    train, validate = train_test_split(train, #second split
                                    test_size=.25, 
                                    random_state=123, 
                                    stratify=train[target_variable])
    return train, validate, test

In [70]:
## acquireing and preparing data for exploration
f = open('data.json')
df = json.load(f)
df = pd.DataFrame(df)

df = w.clean_df(df)

# dropping nulls
df = df.dropna()
df.head()



,repo,language,readme_contents,clean,stemmed,lemmatized
0,DroidPluginTeam/DroidPlugin,Java,Droid Plugin\r\n======\r\n\r\n[中文文档](readme_cn...,droid plugin readmecnmd fllow githubhttpsgithu...,droid plugin readmecnmd fllow githubhttpsgithu...,droid plugin readmecnmd fllow githubhttpsgithu...
1,angular-app/angular-app,JavaScript,[![Build Status](https://secure.travis-ci.org/...,build statushttpssecuretravisciorgangularappan...,build statushttpssecuretravisciorgangularappan...,build statushttpssecuretravisciorgangularappan...
2,doctrine/event-manager,PHP,# Doctrine Event Manager\n\n[![Build Status](h...,doctrine event manager build statushttpsgithub...,doctrin event manag build statushttpsgithubcom...,doctrine event manager build statushttpsgithub...
3,huluoyang/freecodecamp.cn,JavaScript,![](https://s3.amazonaws.com/freecodecamp/wide...,httpss3amazonawscomfreecodecampwidesocialbanne...,httpss3amazonawscomfreecodecampwidesocialbanne...,httpss3amazonawscomfreecodecampwidesocialbanne...
4,spolu/breach_core,JavaScript,### Breach: A Browser for the HTML5 Era\n\n**M...,breach browser html5 era modular everything br...,breach browser html5 era modular everyth brows...,breach browser html5 era modular everything br...


In [77]:
df.language.value_counts

JavaScript      30
Objective-C     18
Java            11
Ruby            11
Python           8
Swift            7
CSS              5
PHP              3
Go               3
TypeScript       2
C++              2
C                2
Rust             2
Haskell          1
CoffeeScript     1
Dart             1
Matlab           1
HTML             1
Kotlin           1
Name: language, dtype: int64

In [82]:
# Assuming you have a DataFrame named 'df' and the column is 'language'
languages = ['JavaScript', 'Objective-C', 'Java', 'Ruby', 'Python', 'Swift']

# Change values to "other" if not in the specified languages
df.loc[~df['language'].isin(languages), 'language'] = 'other'

/var/folders/bf/lhpb5n_j1xj4msxy6nrm9h7h0000gn/T/ipykernel_22443/3120801517.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[~df['language'].isin(languages), 'language'] = 'other'


In [84]:
df.language.unique()

array(['Java', 'JavaScript', 'other', 'Swift', 'Python', 'Objective-C',
       'Ruby'], dtype=object)

In [81]:
df.language.value_counts()

JavaScript     30
Objective-C    18
Java           11
Ruby           11
Python          8
Swift           7
CSS             5
PHP             3
Go              3
TypeScript      2
C++             2
C               2
Rust            2
Name: language, dtype: int64

In [88]:
## splitting data in to train, validate, and test
#train, validate, test = w.split_function(df)
target_variable = "language"
train, validate, test = split_data(df, target_variable)

# A brief look at the data

In [5]:
# train.head()

## A summary of the data

In [6]:
# train.info()

In [7]:
# train.describe

# Explore

## Question 0 (e.g. What is the distribution of the target?)
- NOTE: this question is more of a look at the data, not a stats test question

In [8]:
# Example: # get a boxplot and histplot of property_value of train
## INSERT CODE HERE

## Question 1 (e.g. Is feature_x associated with target?)

In [9]:
# Visualize Example: # get a regplot of squarefeet vs property_value on train
## INSERT CODE HERE

### Analyze feature_x vs target with stats
- $H_0$: There is NO relationship between feature_x and target
- $H_a$: There IS a relationship
- $\alpha$ = .05
    - Example 1: Utilize pearsonr - we are comparing continuous variables, normally distributed
    - Example 2:Comparing a categorical to a continuous variable, normally distributed, and equal variance
        - Testing if property value of has_pool == 1 is signicantly greater than has_pool == 0:  one-tail ttest
        - Utilize f_oneway
    - Many other stats tests

In [10]:
# get the stats from a pearsonr test on feature_x vs target
## INSERT CODE HERE

### Summarize
- p is < $\alpha$, so we can reject the $H_0$ which suggest the $H_a$, i.e. there IS a relationship between feature_x and target (and maybe more description)

## Question 2 (e.g. Is feature_x associated with target?)

In [11]:
# Visualize Example: # get a regplot of squarefeet vs property_value on train
## INSERT CODE HERE

### Analyze feature_x vs target with stats
- $H_0$: There is NO relationship between feature_x and target
- $H_a$: There IS a relationship
- $\alpha$ = .05
    - Example 1: Utilize pearsonr - we are comparing continuous variables, normally distributed
    - Example 2:Comparing a categorical to a continuous variable, normally distributed, and equal variance
        - Testing if property value of has_pool == 1 is signicantly greater than has_pool == 0:  one-tail ttest
        - Utilize f_oneway
    - Many other stats tests

In [12]:
# get the stats from a pearsonr test on feature_x vs target
## INSERT CODE HERE

### Summarize
- p is < $\alpha$, so we can reject the $H_0$ which suggest the $H_a$, i.e. there IS a relationship between feature_x and target (and maybe more description)

## Question 3 (e.g. Is feature_x associated with target?)

In [13]:
# Visualize Example: # get a regplot of squarefeet vs property_value on train
## INSERT CODE HERE

### Analyze feature_x vs target with stats
- $H_0$: There is NO relationship between feature_x and target
- $H_a$: There IS a relationship
- $\alpha$ = .05
    - Example 1: Utilize pearsonr - we are comparing continuous variables, normally distributed
    - Example 2:Comparing a categorical to a continuous variable, normally distributed, and equal variance
        - Testing if property value of has_pool == 1 is signicantly greater than has_pool == 0:  one-tail ttest
        - Utilize f_oneway
    - Many other stats tests

In [14]:
# get the stats from a pearsonr test on feature_x vs target
## INSERT CODE HERE

### Summarize
- p is < $\alpha$, so we can reject the $H_0$ which suggest the $H_a$, i.e. there IS a relationship between feature_x and target (and maybe more description)

## Question 4 (e.g. Is feature_x associated with target?)

In [15]:
# Visualize Example: # get a regplot of squarefeet vs property_value on train
## INSERT CODE HERE

### Analyze feature_x vs target with stats
- $H_0$: There is NO relationship between feature_x and target
- $H_a$: There IS a relationship
- $\alpha$ = .05
    - Example 1: Utilize pearsonr - we are comparing continuous variables, normally distributed
    - Example 2:Comparing a categorical to a continuous variable, normally distributed, and equal variance
        - Testing if property value of has_pool == 1 is signicantly greater than has_pool == 0:  one-tail ttest
        - Utilize f_oneway
    - Many other stats tests

In [16]:
# get the stats from a pearsonr test on feature_x vs target
## INSERT CODE HERE

### Summarize
- p is < $\alpha$, so we can reject the $H_0$ which suggest the $H_a$, i.e. there IS a relationship between feature_x and target (and maybe more description)

## Exploration Summary
* Plain English bullets as required

* Features that have a relationship with target (stats test for feature_y, feature_z, etc. completed on separate working notebook:
    * feature x ...
    * feature y
    * feature z


### (As Required) Features I am moving to modeling with

### (As Required) Features I am not moving to modeling with

# Modeling
* Evaluation Metrics (example: I will use R^2 and Root Mean Square Error (RMSE) as my evaluation metrics)
    * for R^2, the value is in the range 0-1; closer to 1.0 is better; baseline is 0.0
    * for RMSE, the lower the value the better; baseline is xxx
* The average target is xxx which is the baseline prediction
* I will evaluate x different model types and various hyperparameter configurations
    * (Example: The four model types are Ordinary Least Squares (OLS), LassoLars, Polynomial Regression, Generalized Linear Model (GLM))
* Models will be evaluated on train and validate data
* The model that performs the best will then be evaluated on test

In [17]:
## prep data for modeling
## INSERT CODE HERE (examples below

# scaling continuous variable columns for use in modeling
# to_scale = ['bathrooms', 'bedrooms', 'squarefeet', 'lotsize_sqft', 'year']
# train_scaled, validate_scaled, test_scaled = w.get_minmax_scaled(train, validate, test, to_scale)

# get X y splits for modeling
# X_train_scaled, X_validate_scaled, X_test_scaled, y_train, y_validate, y_test = m.get_X_y_baseline(train_scaled, validate_scaled, test_scaled)

## Model Name (example: Ordinary Least Squares (OLS) Regression)

In [18]:
# get model results
# INSERT CODE HERE

* model name did/did not beat baseline

## Model Name (example: LassoLars)

In [19]:
# get model results
# INSERT CODE HERE

* model name did/did not beat baseline

## Model Name (example: Polynomial Regression)

In [20]:
# get model results
# INSERT CODE HERE

* model name did/did not beat baseline

## Model Name (example: Generalized Linear Model (GLM))

In [21]:
# get model results
# INSERT CODE HERE

* model name did/did not beat baseline

## Comparing Models
* summarize the model performance
* ...
* I have chosen to move forward with the ___ model on test

## Best Model (model name) on Test

In [22]:
# get test results for final model
# INSERT CODE HERE

In [23]:
# maybe a visual of model performance (like a residual plot for a regression model)

## Modeling Summary
* summary bullet 1 ...

# Conclusions

## Exploration
* bullets

## Modeling
* bullets

## Recommendations
* bullets

## Next steps
* bullets